In [1]:
import pyrealsense2 as rs
import numpy as np

In [23]:
class py_Realsense():
    def __init__(self):
        self.pipeline = rs.pipeline() # Frame을 받을 pipeline 생성
        self.pc = rs.pointcloud() # Point cloud 생성을 위한 인스턴스 생성
        self.config = rs.config()
        self.align = None
        self.align_to = None

        context = rs.context()
        connect_device = None
        if context.devices[0].get_info(rs.camera_info.name).lower() != 'platform camera':
            connect_device = context.devices[0].get_info(rs.camera_info.serial_number)
        print(" > Serial number : {}".format(connect_device))    

        self.config.enable_device(connect_device)
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)  

        self.align_to = rs.stream.color
        self.align = rs.align(self.align_to)  

    def capture(self):
        # 파이프라인 시작
        self.pipeline.start(self.config)

        frames = self.pipeline.wait_for_frames()
        aligned_frames = self.align.process(frames)

        depth_frame = aligned_frames.get_depth_frame() # 깊이 맵 
        color_frame = aligned_frames.get_color_frame()

        color_image = np.array(color_frame.get_data())

        self.pc.map_to(color_frame)
        points = self.pc.calculate(depth_frame) # 깊이 맵을 포인트 클라우드로 변환

        v = np.array(points.get_vertices()) # 3D 좌표
        c = color_image.reshape(-1, 3)      # 색상 정보

        self.pipeline.stop()

        return v, c, depth_frame

In [109]:
degree = 180
cam = py_Realsense()
vertices, colors, depth_frame = cam.capture()
# vertices : 3D 좌표 정보
# colors : RGB 색상 정보

 > Serial number : 327122076042


In [102]:
print(vertices.shape) # 307200 : 포인트 클라우드의 점의 개수 (x,y,z) 값 나열해서 1차원 배열로 표현
print(colors.shape)
# 이거 viwer 키고 하면 오류남 -> RuntimeError: Frame didn't arrive within 5000

# 깊이 맵을 numpy 배열로 변환
depth_image = np.array(depth_frame.get_data()) # depth_image는 각 픽셀에 대한 거리 정보가 포함된 2D 배열
print(depth_image[1, 1])  # (1, 1) 지점의 깊이 값

(307200,)
(307200, 3)
1393


In [103]:
import open3d as o3d

# 3D 포인트 클라우드 시각화 함수
def visualize_point_cloud(vertices, colors):
    # Open3D PointCloud 객체 생성
    point_cloud = o3d.geometry.PointCloud()
    
    # vertices는 structured array이므로 각 좌표값을 x, y, z로 분리하여 ndarray로 변환
    vertices_array = np.array([vertex.tolist() for vertex in vertices])

    # 3D 좌표와 색상 정보 설정
    point_cloud.points = o3d.utility.Vector3dVector(vertices_array)
    point_cloud.colors = o3d.utility.Vector3dVector(np.asarray(colors, dtype=np.float64) / 255.0)  # 색상 정규화

    
    # 포인트 클라우드 시각화
    o3d.visualization.draw_geometries([point_cloud])

    return point_cloud

In [110]:
pcd = visualize_point_cloud(vertices, colors)
o3d.io.write_point_cloud(f"cloud_{degree}_test1.pcd", pcd)

True